In [1]:
"""
Patterns:
- Mediator Pattern: The mediator centralizes complex communications and controls between objects, 
known as colleagues. In the context of transactions and market participants, the mediator would manage 
the interactions among transactions (e.g., starting, authorizing, and completing transactions) without 
them needing to directly reference each other.
- Observer Pattern: The observer pattern allows objects (observers) to subscribe and react to events 
in other objects (subjects). In our case, transactions would be the subjects, and market participants 
would be the observers. When a transaction changes state (e.g., authorized, completed), 
it would notify all subscribed market participants.


Meditator:
- Transaction Authorization and Cancellation: The Mediator now has a crucial role in deciding whether transactions 
are authorized for execution or need to be cancelled. This decision could be based on more complex logic 
than demonstrated, such as checking against fraud detection algorithms, ensuring sufficient funds, 
or meeting regulatory requirements.
- Dynamic Observer Management: Though not explicitly demonstrated in the code, the Mediator could facilitate 
adding or removing observers from transactions based on certain events or conditions. 
For instance, if a transaction is marked for review, the Mediator could add auditors as observers.
- Centralized Control and Decision-Making: The Mediator's enhanced role demonstrates how it can centralize control 
over transactions' lifecycle, making it easier to implement changes to the process without modifying each transaction class. 
This illustrates the Mediator's value in managing complex interactions and decision-making processes in a system.

PS. In theory, transactions could block each other or may need other for of interaction.
This could be implemented in the Mediator class.

"""

from abc import ABC, abstractmethod
import random

### Observer
class Observer(ABC):
    @abstractmethod
    def update(self, message):
        pass

class MarketParticipant(Observer):
    def __init__(self, name):
        self.name = name
    def update(self, message):
        print(f"{self.name} received notification: {message}")

### Transaction / Subject / Collague
class Transaction(ABC):
    @abstractmethod
    def execute(self):
        pass
    @abstractmethod
    def authorize(self):
        pass
    @abstractmethod
    def cancel(self):
        pass

class Payment(Transaction):
    def __init__(self, id, amount):
        self.id = id
        self.amount = amount
        self.status = "Pending"
        self.observers = []
    def add_observer(self, observer):
        self.observers.append(observer)
    def remove_observer(self, observer):
        self.observers.remove(observer)
    def notify_observers(self, message):
        for observer in self.observers:
            observer.update(message)
    def execute(self):
        print(f"Executing Payment Transaction {self.id} for ${self.amount}")
        self.notify_observers(f"Payment Transaction {self.id} for ${self.amount} executed")
    def authorize(self):
        self.status = "Authorized"
        print(f"Transaction {self.id} authorized.")
        self.notify_observers(f"Transaction {self.id} authorized.")
    def cancel(self):
        self.status = "Cancelled"
        print(f"Transaction {self.id} cancelled.")
        self.notify_observers(f"Transaction {self.id} cancelled.")

### Meditator
class TransactionMediator:
    def __init__(self):
        self.transactions = []
    def add_transaction(self, transaction):
        self.transactions.append(transaction)
    def authorize_transactions(self):
        for transaction in self.transactions:
            # Randomly authorize transactions for demonstration
            authorized = random.choice([True, False])  
            if authorized:
                transaction.authorize()
            else:
                transaction.cancel()
    def execute_authorized_transactions(self):
        for transaction in self.transactions:
            if transaction.status == "Authorized":
                transaction.execute()

In [2]:
### Client code
mediator = TransactionMediator()

### Creating transactions
transaction1 = Payment("TX_01", 100)
transaction2 = Payment("TX_02", 200)

### Adding transactions to mediator
mediator.add_transaction(transaction1)
mediator.add_transaction(transaction2)

### Creating market participants and subscribing to transactions
market_participant_a = MarketParticipant("A")
market_participant_b = MarketParticipant("B")

transaction1.add_observer(market_participant_a)
transaction2.add_observer(market_participant_a)
transaction2.add_observer(market_participant_b)

### Mediator authorizes and executes transactions
mediator.authorize_transactions()
mediator.execute_authorized_transactions()

Transaction TX_01 authorized.
A received notification: Transaction TX_01 authorized.
Transaction TX_02 cancelled.
A received notification: Transaction TX_02 cancelled.
B received notification: Transaction TX_02 cancelled.
Executing Payment Transaction TX_01 for $100
A received notification: Payment Transaction TX_01 for $100 executed
